In [1]:
import requests
import base64
import json
import cv2
import numpy as np
import glob
import os
import time
import pandas as pd
import re

# Google doc ai function defined

In [2]:
def gv_doc_ai_data_extractor(file_path,doctype):
    
    url_address = "https://uipathmldev.corp.uber.com"
    #url_address = "http://127.0.0.1:5000"
    # url_address = http://DCA1-UP-ML-P01
    # url_address = "http://dca1-ic-d-aby01:5000/iacoe_ocr/doc-ai/data_extractor"
    ENDPOINT = '/iacoe_ocr/doc-ai/data_extractor_v1'
    url = url_address + ENDPOINT
   
    with open(file_path, 'rb')as file_obj:
        file = file_obj.read()
    file_base64 = base64.b64encode(file)

    payload = {'file_base64': file_base64,
               'doctype':doctype,
    }
    print('j')
    response = requests.post(url, data=payload, verify=False)
    print(response)
    json_data = json.loads(response.text.encode('utf-8'))
    
    print('j')
    doc_ai_base64 = json_data['success']
    
    doc_ai_base64 = eval(doc_ai_base64)
    
    doc_ai_bytes = base64.b64decode(doc_ai_base64)
    doc_ai_json = json.loads(doc_ai_bytes)
    return doc_ai_json

# Saving output in json format in local

In [26]:
DATASET_INP_PATH = r'/Users/jghosh2/Documents/my-notebook/INVOICE-PROCESSING/data/apac_nonenglish'
dirs = os.listdir(DATASET_INP_PATH)
print(dirs)
doctype = 'p2p_invoice_trials'
data_files = {}
for class_folder in dirs:
    if class_folder!=".DS_Store":
        class_folder_path = os.path.join(DATASET_INP_PATH, class_folder)
        files=glob.glob(class_folder_path+'/*.pdf')
        try:
            for i,doc_path in enumerate(files):
                filenames=doc_path.split('/')[-1][:-4]
                t1 = time.time()
                doc_ai_json= gv_doc_ai_data_extractor(doc_path,doctype)  
                f_ext=".json"
                new_name = f'{filenames}{f_ext}'
                with open(os.path.join(class_folder_path,new_name),'w') as f:  
                    json.dump(str(doc_ai_json), f, indent = 6)    
                print("File successfully converted-{}".format(new_name))
        except:
            pass

['.DS_Store', '1045939 - GOLDENCODE CO LTD', '1052531 - EDGERANK INC']
j


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uipathmldev.corp.uber.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>
j
File successfully converted-027_TC26524856.json
j


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uipathmldev.corp.uber.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>
j


# Storing all data in a dataframe

In [27]:
#DATASET_INP_PATH = r'/Users/jghosh2/Documents/my-notebook/INVOICE-PROCESSING/data/table_testing_apac'
DATASET_INP_PATH = r'/Users/jghosh2/Documents/my-notebook/INVOICE-PROCESSING/data/FTesting/set10'
dirs = os.listdir(DATASET_INP_PATH)
dataset = []
for class_folder in dirs:
    if class_folder != ".DS_Store":
        class_folder_path = os.path.join(DATASET_INP_PATH, class_folder)
        #class_files = os.listdir(class_folder_path)
        files=glob.glob(class_folder_path+'/*.json')       
        for filepath in files:
            if filepath != ".DS_Store":
                with open(filepath, 'r') as f:
                    raw_json = eval(json.loads(f.read()))
                data = [filepath.split('/')[-1], raw_json, class_folder]
                dataset.append(data)

# Function for extracting entities from json

In [144]:
def get_default_format(keys, pages_dimensions):
    entities_data = {}
    default_width = pages_dimensions["width"]
    default_height = pages_dimensions["height"]
    default_data = {
        "text" : "NA",
        "conf_score" : "NA",
        "coordinates" : [
            {'x': 0, 'y': 0}, 
            {'x': default_width, 'y': 0}, 
            {'x': default_width, 'y': default_height}, 
            {'x': 0, 'y': default_height}
        ],
        "page_idx": "NA"
    }
    for key in keys:
        entities_data[key] = default_data.copy()
    return entities_data
def entities(data):
    
    # OUTPUT KEYS
    SUPPLIER_OUT_KEY = "Supplier"
    ENTITY_OUT_KEY = "Legal Entity"
    INV_NO_OUT_KEY = "Invoice Num"
    INV_DATE_OUT_KEY = "Invoice Date"
    PO_NUMBER_OUT_KEY = "PO Number"
    SERVICE_START_DATE_OUT_KEY = "Service Start Date"
    SERVICE_END_DATE_OUT_KEY = "Service End Date"
    CURRENCY_OUT_KEY = "Invoice Curr"
    TAX_OUT_KEY = "Tax Amount"
    TAX_RATE_OUT_KEY = "Tax Rate"
    TERMS_OUT_KEY = "Terms"
    SUB_TOTAL_OUT_KEY = "Line Total/Subtotal"
    TOTAL_OUT_KEY = "Invoice Amount"
    REMIT_ADDRESS_OUT_KEY = "Remit Address"

    # STATUS FLAGS
    INV_NO_OUT_KEY_FLAG = True
    INV_DATE_OUT_KEY_FLAG = True
    SUPPLIER_OUT_KEY_FLAG = True
    PO_NUMBER_OUT_KEY_FLAG = True
    CURRENCY_OUT_KEY_FLAG = True
    TAX_OUT_KEY_FLAG = True
    TAX_RATE_OUT_KEY_FLAG = True
    TERMS_OUT_KEY_FLAG = True
    SUB_TOTAL_OUT_KEY_FLAG = True
    TOTAL_OUT_KEY_FLAG = True
    SERVICE_START_DATE_OUT_KEY_FLAG=True
    SERVICE_END_DATE_OUT_KEY_FLAG=True
    
    OUT_KEYS = [SUPPLIER_OUT_KEY, ENTITY_OUT_KEY, INV_NO_OUT_KEY, INV_DATE_OUT_KEY, PO_NUMBER_OUT_KEY, SERVICE_START_DATE_OUT_KEY,
               SERVICE_END_DATE_OUT_KEY, CURRENCY_OUT_KEY, TAX_OUT_KEY, TAX_RATE_OUT_KEY, TERMS_OUT_KEY, SUB_TOTAL_OUT_KEY,
               TOTAL_OUT_KEY, REMIT_ADDRESS_OUT_KEY]

    #data = json.loads(data)
    raw_text, pages = data['raw_text'], data['pages']
    forms_data, tables_data = pages[0]['forms_data'], pages[0]['tables_data']
    pages_dimension = pages[0]["dimension"]

    parsed_data = get_default_format(OUT_KEYS, pages_dimension)
    po_list=['Order Ref\n','OSCA / PWD IDNO.:','✓ OSCA/PWD IDNo.:','P.O./S.O. Number:','Client Reference',
             'PO ORDER NO:','OSCA/PWD IDNo.:','ORDER NO:','Client Reference:','PO# -','PO:','PO','Reference',
             'Order No','PO NO','to','P. O No','Client PO','P. Order No','PO#','Customer PO','PO Number',
             'Order form','Customer PO No','PO No','P/O Number','(Your Reference','CUST ORDER NO',
             'Purchase Order','CUSTOMER PO NO']
    invoice_date_list=['25','B:','Document Date','DATE','Invoice date','Emission date','Date','Bill Date','Billing Date\n','Invoice Date',
                       'INVOICE DATE','Invoice date']
    invoice_no_list=['YU','TAX INVOICE','Document Number','INVOICE No.','Credit Note No.','Debit Note No.','INVOICE NUMBER','Billing Number','Billing No','Invoice number','Document No',
                     'Tax Invoice','Invoice Number','Invoice #','Billing Statement No.','Bill No','INVOICE NO',
                     'Number:','Invoice number','Invoice No','Electronic Export Invoice','NO']
    subtotal_list=['*R','A:\n','Total Amount:','BALANCE DUE','Sub Total:','Total Ex','Total Before Tax:','SUBTOTAL:','Cost Total (excl. tax)',
                   'Voubtotal:','Premium','Total:','Total Net Amount:','Subtotal:',
                   'Total Charges for Bill Period','NET AMOUNT','Total Value','Service Fees','Item Total',
                   'Invoice Subtotal:','Total value','Sub Total','Total amount owing (exc GST) NZD',
                   'Cost Total (excl','Total Net Amount',
                   'Subtotal','Amount Due','Net','TWD']
    invoice_amount_list=['Total Amount:','TOTAL USD','AUD','BALANCE DUE','Total AUD','Amount Due(LKR)',
                         'TOTAL:','Total Amount','Thank You!','Total:','Cost Total (incl. tax)','Balance Due:',
                         'TOTAL','Total amount','TOTAL CREDIT','TOTAL PHP','Invoice Total ( JPY):',
                         'Invoice Total (JPY):','Grand Total with VAT','Invoice Amount:','Grand Total',
                         'TOTAL AMOUNT\nAUD','Total amount owing (incl GST) NZD','Total Charges for Bill Period',
                         'TOTAL AMOUNT','Total Amount Due:','/ Total Amount','THAN / Total Amount',
                         'Invoice Amount','Invoice Total:','Balance Payment','TOTAL DUE','Total',
                         'Cost Total (incl','TOTAL AUD','Invoice Total Due','Total Amount Payable',
                         'Invoice Total','Total Recurring Charges', 'Net Amount','NET AMOUNT',
                         'Fixed Fee','Amount Due']
    tax_amount_list=['6:','BAR:','Tax','Tax (10%)','GST','VAT @ 15%','TOTAL TAX','VAT','GST:','VAT (7.5%)','12% VAT','VAT 8%','Total GST on invoice NZD',
                     'Tax (Cost Total X 10%)','VAT (8.00%)','Total Tax Amount:','GST AMOUNT','TOTAL GST 10%']
    def get_supplier_legal_entity(raw_text):
        '''Function to extract supplier and Legal entites from raw_text'''
        #L&L, Leaven & Co., CPAs
        if raw_text.find('L&L, Leaven & Co., CPAs') != -1:
            supplier = 'L&L, Leaven & Co., CPAs'
            legal_entity = 'TAIWAN YUBO CO., LTD.'
        #51201 - JONES LANG LASALLE LANKA PVT LTD
        elif raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd') != -1:
            supplier = 'Jones Lang Lasalle Lanka (Private) Ltd'
            legal_entity = 'Uber Lanka (Private) Limited'
        #37889 - CDW IT SOLUTIONS AUSTRALIA PTY LTD
        elif raw_text.find('CDW IT Solutions Australia PTY LTD') != -1 :
            supplier = 'CDW IT Solutions Australia PTY LTD'
            legal_entity = 'Uber Australia Pty Ltd'
        #42834 - CDW IT SOLUTIONS SINGAPORE PTE LTD
        elif raw_text.find('CDW IT Solutions Singapore PTE. Limited') != -1:
            supplier = 'CDW IT Solutions Singapore PTE. Limited'
            legal_entity = 'Uber Singapore Technology Pte. Ltd'
         #27561 - EQUINIX INC   
        elif raw_text.find('Equinix Singapore Pte Ltd') != -1:
            supplier = 'Equinix Singapore Pte Ltd'
            legal_entity = 'Uber Singapore Technology Pte. LTD.'
        #Mobitel (Pvt) Ltd
        elif raw_text.find('Mobitel (Pvt) Ltd') != -1:
            supplier = 'Mobitel'
            legal_entity = 'Uber Lanka Pvt Ltd'
        #SK Solutions
        elif raw_text.find('SK Solutions') != -1:
            supplier = 'SK SOLUTIONS'
            legal_entity = 'Uber Bangladesh Ltd.'
        #Business Process Outsourcing International
        elif raw_text.find('Business Process Outsourcing International') != -1:
            supplier = 'Business Process Outsourcing International'
            legal_entity = 'Uber Philippines Centre of Excellence Limited Liability Company'
        #EOS IT MANAGEMENT SOLUTIONS INC
        elif raw_text.find('EOS IT MANAGEMENT SOLUTIONS INC') != -1:
            supplier = 'EOS IT MANAGEMENT SOLUTIONS INC'
            legal_entity = 'UBER TECHNOLOGIES INC'
        #POWERNET IT SOLUTIONS
        elif raw_text.find('PowerNET Group Pty Ltd') != -1:
            supplier = 'PowerNET Group Pty Ltd'
            legal_entity = 'UBER AUSTRALIA PTY LTD'
        #FUJI XEROX PHILS. INC.
        elif raw_text.find('FUJIFILM Business Innovation Philippines Corp.') != -1:
            supplier = 'FUJIFILM Business Innovation Philippines Corp.'
            legal_entity = 'UBER PHILIPPINES CENTER OF EXCELLENCE LIMITED LIABILITY COMPANY'
        #MEERO SAS
        elif raw_text.find('Meero SAS') != -1:
            supplier = 'Meero SAS'
            legal_entity = 'Uber Australia PTY LTD'
        elif raw_text.find('Bespoke Nominees Pty Ltd') != -1:
            supplier = 'Bespoke Nominees Pty Ltd'
            legal_entity = 'Uber Australia Pty Ltd'
        elif raw_text.find('HAKUHODO') != -1:
            supplier = 'HAKUHODO'
            legal_entity = 'Uber Eats Japan'
        elif raw_text.find('Detmold New Zealand Sales Ltd') != -1:
            supplier = 'Detmold New Zealand Sales Ltd'
            legal_entity = 'Uber New Zealand Technologies LTD'
        elif raw_text.find('ADA Digital Singapore Pte Ltd') != -1:
            supplier = 'ADA Digital Singapore Pte Ltd'
            legal_entity = 'UBER LANKA (Pvt) Ltd'
        elif raw_text.find('S.E. RENTALS PTY LTD') != -1:
            supplier = 'S.E. RENTALS PTY LTD'
            legal_entity = 'UBER Australia Pty Ltd'
        elif raw_text.find('Next Online Limited') != -1:
            supplier = 'Next Online Limited'
            legal_entity = 'Uber Bangladesh Ltd'
        elif raw_text.find('Telstra International Limited') != -1:
            supplier = 'Telstra International Limited'
            legal_entity = 'NA'
        elif raw_text.find('Leaf d Private Limited') != -1:
            supplier = 'Leaf d Private Limited'
            legal_entity = 'Uber Lanka (Private) Limited'
        elif raw_text.find('KPMG') != -1:
            supplier = 'KPMG'
            legal_entity = 'Uber Portier Taiwan Co. Ltd.'
        elif raw_text.find('GroupM Japan K.K.') != -1:
            supplier = 'GroupM Japan K.K.'
            legal_entity = 'Uber Eats Japan, Inc.'
        elif raw_text.find('KMC Mag Solutions, Inc.') != -1:
            supplier = 'KMC Mag Solutions, Inc.'
            legal_entity = 'Uber Philippines Centre of Excellence LLC'
        elif raw_text.find('MEDIACOM') != -1 & raw_text.find('GroupM Japan K.K.') == -1:
            supplier = 'MEDIACOM'
            if raw_text.find('UBER AUSTRALIA PTY LTD') != -1:
                legal_entity = 'UBER AUSTRALIA PTY LTD'
            else:
                legal_entity = 'UBER PORTIER B.V.'
        elif raw_text.find('Dialog Bredbandi betwerks DNA LEN Ý') != -1:
            supplier = 'Dialog Bredbandi betwerks DNA LEN Ý'
            legal_entity = 'UBER LANKA (PRIVATE) LIMITED'
        elif raw_text.find('Giftaway, Inc.') != -1:
            supplier = 'Giftaway, Inc.'
            legal_entity = 'Uber Philippines Center of Excellence'
        elif raw_text.find('Jones Lang LaSalle Bangladesh Pvt. Ltd.') != -1:
            supplier = 'Jones Lang LaSalle Bangladesh Pvt. Ltd.'
            legal_entity = 'Uber Bangladesh Limited.'
        elif raw_text.find('Quess Corp Lanka Private Limited') != -1:
            supplier = 'Quess Corp Lanka Private Limited.'
            legal_entity = 'Uber Lanka (Private) Limited'
        elif raw_text.find('Aon Singapore Pte. Ltd.') != -1:
            supplier = 'Aon Singapore Pte. Ltd.'
            legal_entity = 'Uber Singapore Technology Pte. Ltd.'
        elif raw_text.find('EPI ET RIZ K.K.') != -1:
            supplier = 'EPI ET RIZ K.K.'
            legal_entity = 'Uber Japan Co., LTD.'
        elif raw_text.find('HARDTALK (PVT) LTD') != -1:
            supplier = 'HARDTALK (PVT) LTD'
            legal_entity = 'Uber Lanka (Private) Limited'
        elif raw_text.find('Special Group Australia Pty Ltd') != -1:
            supplier = 'Special Group Australia Pty Ltd'
            legal_entity = 'Uber Australia PTY Ltd'
        elif raw_text.find('Carripook Interior Plant Care') != -1:
            supplier = 'Carripook Interior Plant Care'
            legal_entity = 'UBER AUSTRALIA PTY LTD'
        elif raw_text.find('Computec Engineering, Ltd') != -1:
            supplier = 'Computec Engineering, Ltd'
            legal_entity = 'Uber Japan Co., LTD.'
        elif raw_text.find('adplanet communication') != -1:
            supplier = 'adplanet communication'
            legal_entity = 'UT LLC'
        elif raw_text.find('All Branded Group Pty Limited') != -1:
            supplier = 'All Branded Group Pty Limited'
            legal_entity = 'Uber Australia PTY LTD'
        elif raw_text.find('Kavindu Sankalpa Gunawardana') != -1:
            supplier = 'Kavindu Sankalpa Gunawardana'
            legal_entity = 'Uber lanka private limited'
        elif raw_text.find('KIAN-KIEFER ENTERPRISES') != -1:
            supplier = 'KIAN-KIEFER ENTERPRISES'
            legal_entity = 'Upper Philippines Center excellence'
        elif raw_text.find('Superior Air Conditioning') != -1:
            supplier = 'Superior Air Conditioning'
            legal_entity = 'Uber Australia Pty Ltd'
        elif raw_text.find('HH Australia Pty Ltd') != -1:
            supplier = 'HH Australia Pty Ltd'
            legal_entity = 'Uber Australia PTY LTD'
        elif raw_text.find('AMBAGAHA DEPELE GEDARA JAGATH KELUM DISSANAYAKE') != -1:
            supplier = 'AMBAGAHA DEPELE GEDARA JAGATH KELUM DISSANAYAKE'
            legal_entity = 'Uber Lanka (Private) Limited'            
        elif raw_text.find('First Aurora Limited (HK)') != -1:
            supplier = 'First Aurora Limited (HK)'
            legal_entity = 'Taiwan Yubo Co Ltd'
        elif raw_text.find('Yakusan Pty Ltd') != -1:
            supplier = 'Yakusan Pty Ltd'
            legal_entity = 'Uber Australia Pty Ltd'
        elif raw_text.find('INXPRESS AUSTRALIA') != -1:
            supplier = 'INXPRESS AUSTRALIA'
            legal_entity = 'UBER AUSTRALIA PTY LTD'
        elif raw_text.find('Consolidated Office Choice') != -1:
            supplier = 'Consolidated Office Choice'
            legal_entity = 'UBER AUSTRALIA PTY LTD'
        elif raw_text.find('Hawker Britton Group Pty Ltd') != -1:
            supplier = 'Hawker Britton Group Pty Ltd'
            legal_entity = 'UBER AUSTRALIA PTY LTD'
        elif raw_text.find('PCCWⓇ') != -1:
            supplier = 'PCCWⓇ'
            legal_entity = 'UberEats Management Taiwan Co. Ltd'
        elif raw_text.find('mtc co.') != -1:
            supplier = 'mtc co. ltd'
            legal_entity = 'Uber Japan Co., LTD.'
        elif raw_text.find('Yahoo Japan Corporation') != -1:
            supplier = 'Yahoo Japan Corporation'
            legal_entity = 'Uber Japan£H'
        elif raw_text.find('BLUE WIRELESS PTE. LTD.') != -1:
            supplier = 'BLUE WIRELESS PTE. LTD.'
            legal_entity = 'Uber Eats Japan, Inc.' 
        elif raw_text.find('Aegis BPO Malaysia Sdn Bhd') != -1:
            supplier = 'Aegis BPO Malaysia Sdn Bhd'
            legal_entity = 'UT LLC'
        elif raw_text.find('Apple Distribution International Ltd.') != -1:
            supplier = 'Apple Distribution International Ltd.'
            legal_entity = 'UT LLC'
        elif raw_text.find('89612904') != -1:
            supplier = '89612904'
            legal_entity = '91111080' 
        elif raw_text.find('70503981') != -1:
            supplier = '70503981'
            legal_entity = 'Uber Eats' 
        else:
            supplier = 'NA'
            legal_entity = 'NA'       
        return (supplier, legal_entity)
    def get_po_no(raw_text):
        if raw_text.find('Business Process Outsourcing International') != -1 and raw_text.find('Address')!= -1:
            p=raw_text.find('PARTICULARS')
            x=raw_text[p:p+150]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('CDW IT Solutions Australia PTY LTD')!= -1:
            p=raw_text.find('CUST ABN NO')
            x=raw_text[p-60:p]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('Bespoke Nominees Pty Ltd')!= -1:
            p=raw_text.find('Reference')
            x=raw_text[p:p+100]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('Telstra International Limited')!= -1 or raw_text.find('EPI ET RIZ K.K.')!= -1  or  raw_text.find('Mobitel')!= -1 or raw_text.find('KPMG')!= -1 :             
            p=raw_text.find('PO#')
            x=raw_text[p:p+50]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('KMC Mag Solutions, Inc.')!= -1:
            p=raw_text.find('PO #')
            x=raw_text[p:p+50]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('Leaf d Private Limited')!= -1:
            p=raw_text.find('PO-')
            x=raw_text[p+2:p+50]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
        elif raw_text.find('Kavindu Sankalpa Gunawardana')!= -1:
            p=raw_text.find('PO Number')
            x=raw_text[p+2:p+50]
            match = re.search(r'\d{6}', x) 
            if match!=None:
                po_no=match[0]
            else:
                po_no='NA'
                
        elif raw_text.find('INXPRESS AUSTRALIA')!= -1:
            p=raw_text.find('BILL TO:')
            x=raw_text[p:p+100]
            po_no=x.split('\n')[2]
            
        elif raw_text.find('Consolidated Office Choice')!= -1:
            p=raw_text.find('Order Ref')
            x=raw_text[p:p+100]
            po_no=x.split('\n')[6]  
        elif raw_text.find('BLUE WIRELESS PTE. LTD.')!= -1:   
            p=raw_text.find('PO#')
            x = raw_text[p:p+100]
            po_no=x.split('\n')[0]
        elif raw_text.find('PCCWⓇ')!= -1 or raw_text.find('mtc co.')!= -1 or  raw_text.find('Yahoo Japan Corporation')!= -1:    
            p=raw_text.find('PO')
            x=raw_text[p:p+50]
            po_no=x.split('\n')[0]
        elif raw_text.find('Apple Distribution International Ltd.')!= -1 :    
            p=raw_text.find('PO #')
            x=raw_text[p:p+50]
            po_no=x.split('\n')[0]   
            
        elif raw_text.find('89612904')!= -1 :    
            p=raw_text.find('po:')
            x=raw_text[p:p+50]
            po_no=x.split('\n')[0]      

        else:
            po_no='NA'
        return po_no
    def get_invoice_date(raw_text):
        if raw_text.find('Business Process Outsourcing International') != -1 and raw_text.find('Address')!= -1:
            p=raw_text.find('Address')
            x=raw_text[p:p+150].replace('hm','Jun')
            match = re.search(r'\d{2}-(hm|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)-\d{2}', x)
            if match!=None:
                invoice_date=match[0]
            else:
                invoice_date='NA'
        elif raw_text.find('HAKUHODO') != -1 :
            p=raw_text.find('HAKUHODO')
            x=raw_text[p:p+150]
            match = re.search(r'\d{4}.\d{2}.\d{2}',x)
            if match!=None:
                invoice_date=match[0]
            else:
                invoice_date='NA'
        elif raw_text.find('PowerNET Group Pty Ltd') != -1:
            start_key = 'Date'
            sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+50]
            match = re.findall('\d+[/]\d+[/]\d{4}', sub_string)
            if match!=None:
                invoice_date=match[0]
            else:
                invoice_date='NA'
        elif raw_text.find('Detmold New Zealand Sales Ltd') != -1:
            p=raw_text.find('Invoice Date')
            x=raw_text[p:p+100]
            invoice_date=x.split('\n')[2]
        elif raw_text.find('All Branded Group Pty Limited') != -1:
            p=raw_text.find('DATE')
            x=raw_text[p:p+80]
            invoice_date=x.split('\n')[1]
        elif raw_text.find('Superior Air Conditioning')!= -1:
            p=raw_text.find('Tax Invoice')
            x=raw_text[p:p+100]
            invoice_date=x.split('\n')[3]
        else:
            invoice_date='NA'
        
        return invoice_date
    def get_invoice_no(raw_text):
        if raw_text.find('Meero SAS') != -1 :
            p=raw_text.find('FA')
            invoice_no=raw_text[p:p+23]
        elif  raw_text.find('EPI ET RIZ K.K.') != -1 :    
            p=raw_text.find('No,')
            x = raw_text[p:p+50]
            match = re.search(r'\d{7}', x) 
            if match!=None:
                invoice_no=match[0]
            else:
                invoice_no='NA'
        elif  raw_text.find('All Branded Group Pty Limited') != -1 :    
            p=raw_text.find('INVOICE #')
            x = raw_text[p:p+50]
            match = re.search(r'\d{5}', x) 
            if match!=None:
                invoice_no=match[0]
            else:
                invoice_no='NA'
        elif  raw_text.find('89612904') != -1 :    
            p=raw_text.find('TC')
            x = raw_text[p:p+50]
            match = re.search(r'\d+', x) 
            if match!=None:
                invoice_no=match[0]
            else:
                invoice_no='NA'       
        else:
            invoice_no='NA'
        return invoice_no
    def get_subtotal(raw_text):
        if raw_text.find('Meero SAS') != -1 :
            p=raw_text.find('Total (tax excluded)\n')
            x=raw_text[p:p+100]
            subtotal=x.split('\n')[1].replace('$','')
        elif raw_text.find('Telstra International Limited')!= -1:
            p=raw_text.find('\nAmount\n(USD)\n')
            x=raw_text[p:p+50]
            subtotal=x.split('\n')[3] 
        elif raw_text.find('MEDIACOM') != -1 and raw_text.find('UBER AUSTRALIA PTY LTD') != -1:
            p=raw_text.find('Total Billable:')
            x = raw_text[p-40:p]
            subtotal=x.split('\n')[2] 
        elif raw_text.find('Mobitel ') != -1 :
            p=raw_text.find('\nCess ')
            x = raw_text[p-40:p]
            subtotal=x.split('\n')[-1] 
        elif raw_text.find('GroupM Japan K.K. ') != -1 :
            p=raw_text.find('Subtotal')
            x = raw_text[p:p+100]
            subtotal=x.split('\n')[3] 
        elif raw_text.find('L&L, Leaven & Co., CPAS') != -1:
            p=raw_text.find('Please pay within 30 days')
            x = raw_text[p-50:p]
            subtotal=x.split('\n')[-2]
        elif raw_text.find('FUJIFILM Business Innovation Philippines Corp.') != -1 :
            p=raw_text.find('AMOUNT')
            x = raw_text[p:p+200]
            subtotal=x.split('\n')[4]  
        elif raw_text.find('Kavindu Sankalpa Gunawardana') != -1:
            p=raw_text.find('Total')
            x = raw_text[p:p+60]
            subtotal=x.split('\n')[1]
        elif raw_text.find('All Branded Group Pty Limited') != -1:
            p=raw_text.find('Pay by Credit')
            x = raw_text[p-100:p]
            subtotal=x.split('\n')[-5]
        elif raw_text.find('Hawker Britton Group Pty Ltd') != -1: 
            p=raw_text.find('Gross')
            x = raw_text[p:p+100]
            subtotal=x.split('\n')[5]
        elif raw_text.find('Yahoo Japan Corporation') != -1:     
            p=raw_text.find('Invoice Total')
            x=raw_text[p:p+100]
            subtotal=x.split('\n')[3]
        elif raw_text.find('Aegis BPO Malaysia Sdn Bhd') != -1:      
            p=raw_text.find('Total due')
            x=raw_text[p:p+100]
            subtotal=x.split('\n')[1]
        else:
            subtotal='NA'
        return subtotal
    def get_invoice_amount(raw_text):
        if raw_text.find('Telstra International Limited')!= -1:
            p=raw_text.find('Total Amount\n')
            x=raw_text[p:p+50]
            invoice_amount=x.split('\n')[2]
        elif raw_text.find('KPMG')!= -1:
            p=raw_text.find('Total Amount\n')
            x=raw_text[p:p+50]
            invoice_amount=x.split('\n')[2]
        elif raw_text.find('S.E. RENTALS PTY LTD')!= -1:
            p=raw_text.find('\nMonthly')
            x=raw_text[p:p+50]
            invoice_amount=x.split('\n')[4].replace('$','')
        elif raw_text.find('MEDIACOM') != -1 and raw_text.find('UBER AUSTRALIA PTY LTD') != -1:
            p=raw_text.find('Amount (USD)')
            x = raw_text[p:p+40]
            invoice_amount=x.split('\n')[3] 
        elif raw_text.find('CDW IT Solutions Singapore PTE. Limited') != -1:
            p=raw_text.find('TOTAL AMOUNT')
            x = raw_text[p:p+100]
            invoice_amount=x.split('\n')[2]
        elif raw_text.find('Meero SAS') != -1:
            p=raw_text.find('Total (incl tax)')
            x = raw_text[p:p+100]
            invoice_amount=x.split('\n')[1].replace('$','')
        elif raw_text.find('Jones Lang LaSalle Bangladesh Pvt. Ltd.') != -1:
            p=raw_text.find('In Word:')
            x = raw_text[p-100:p]
            invoice_amount=x.split('\n')[-2]
        elif raw_text.find('Quess Corp Lanka Private Limited') != -1:
            p=raw_text.find('Invoice Amount')
            x = raw_text[p:p+200]
            invoice_amount=x.split('\n')[3]
        elif raw_text.find('Kavindu Sankalpa Gunawardana') != -1:
            p=raw_text.find('Total')
            x = raw_text[p:p+60]
            invoice_amount=x.split('\n')[1]
        elif raw_text.find('All Branded Group Pty Limited') != -1:
            p=raw_text.find('Pay by Credit')
            x = raw_text[p-100:p]
            invoice_amount=x.split('\n')[-3]
        elif raw_text.find('Yahoo Japan Corporation') != -1:     
            p=raw_text.find('Invoice Total')
            x=raw_text[p:p+100]
            invoice_amount=x.split('\n')[1]
        else:
            invoice_amount='NA'
        return invoice_amount
    
    def get_currency(raw_text):
        '''Function to extract currency from raw_text'''
        if raw_text.find('L&L, Leaven & Co., CPAs') != -1 or raw_text.find('KPMG') != -1:
            currency = 'TWD'
        elif raw_text.find('AMBAGAHA DEPELE GEDARA JAGATH KELUM DISSANAYAKE') != -1 or raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd') != -1 or raw_text.find('Quess Corp Lanka Private Limited') != -1 or raw_text.find('ADA Digital Singapore Pte Ltd') != -1 :
            currency = 'LKR'
        elif raw_text.find('BLUE WIRELESS PTE. LTD.') != -1 or raw_text.find('First Aurora Limited (HK)') != -1 or raw_text.find('Telstra International Limited') != -1  or raw_text.find('Equinix Singapore Pte Ltd') != -1 or raw_text.find('EOS IT MANAGEMENT SOLUTIONS INC') != -1:
            currency = 'USD'
        elif raw_text.find('CDW IT Solutions Singapore PTE. Limited') != -1 or raw_text.find('Aon Singapore Pte. Ltd.') != -1:
            currency = 'SGD'
        elif raw_text.find('SK Solutions') != -1 or  raw_text.find('Jones Lang LaSalle Bangladesh Pvt. Ltd.') != -1 or  raw_text.find('Next Online Limited') != -1:
            currency = 'BDT'        
        elif raw_text.find('Giftaway, Inc.') != -1 or raw_text.find('Business Process Outsourcing International') != -1 or raw_text.find('KMC Mag Solutions, Inc.') != -1:
            currency = 'PHP'
        elif raw_text.find('Hawker Britton Group Pty Ltd') != -1 or raw_text.find('Yakusan Pty Ltd') != -1 or raw_text.find('HH Australia Pty Ltd') != -1 or raw_text.find('CDW IT Solutions Australia PTY LTD') != -1 or raw_text.find('PowerNET Group Pty Ltd') != -1 or raw_text.find('Bespoke Nominees Pty Ltd') != -1:
            currency = 'AUD'
        elif raw_text.find('FUJIFILM Business Innovation Philippines Corp.') != -1:
            currency = 'Php'
        elif raw_text.find('INXPRESS AUSTRALIA') != -1 or raw_text.find('Superior Air Conditioning') != -1  or raw_text.find('Meero SAS') != -1 or raw_text.find('All Branded Group Pty Limited') != -1 or raw_text.find('Carripook Interior Plant Care') != -1 or raw_text.find('Special Group Australia Pty Ltd') != -1 or raw_text.find('S.E. RENTALS PTY LTD')!= -1 :
            currency = '$'
        elif raw_text.find('mtc co.') != -1 or raw_text.find('HAKUHODO') != -1 or raw_text.find('EPI ET RIZ K.K.') != -1:
            currency = '￥'
        elif raw_text.find('Yahoo Japan Corporation') != -1  or raw_text.find('GroupM Japan K.K.') != -1 or raw_text.find('Computec Engineering, Ltd') != -1:
            currency = 'JPY'
        elif raw_text.find('Detmold New Zealand Sales Ltd') != -1:
            currency = 'NZD'
        elif raw_text.find('HARDTALK (PVT) LTD') != -1 or raw_text.find('Dialog Bredbandi betwerks DNA LEN Ý') != -1:
            currency = 'Rs.'
        elif raw_text.find('Apple Distribution International Ltd.') != -1 or raw_text.find('Aegis BPO Malaysia Sdn Bhd') != -1 or raw_text.find('PCCWⓇ') != -1 or raw_text.find('MEDIACOM') != -1 and raw_text.find('GroupM Japan K.K.') == -1:
            currency = 'USD'
        elif raw_text.find('adplanet communication') != -1:
            currency = 'KRW'
    
        else:
            currency = 'NA'
        return currency
    def get_taxrate(raw_text):
        '''Function to extract taxrate from raw_text'''
        if  raw_text.find('mtc co.') != -1 or raw_text.find('Yahoo Japan Corporation') != -1 or\
        raw_text.find('Hawker Britton Group Pty Ltd') != -1 or\
        raw_text.find('Yakusan Pty Ltd') != -1 or raw_text.find('HH Australia Pty Ltd') != -1 or\
        raw_text.find('EPI ET RIZ K.K.') != -1 or raw_text.find('CDW IT Solutions Australia PTY LTD') != -1 or\
        raw_text.find('GroupM Japan K.K.') != -1  or raw_text.find('HAKUHODO') != -1 or\
        raw_text.find('Bespoke Nominees Pty Ltd') != -1:
            taxrate = '10.00' 
        elif raw_text.find('Equinix Singapore Pte Ltd') != -1 or raw_text.find('MEDIACOM') != -1 or raw_text.find('CDW IT Solutions Singapore PTE. Limited') != -1 :
            taxrate = '7.00'
        elif raw_text.find('SK Solutions') != -1: 
            taxrate = '7.5'        
        elif raw_text.find('Mobitel (Pvt) Ltd') != -1  or raw_text.find('Quess Corp Lanka Private Limited') != -1 or raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd') != -1 or raw_text.find('ADA Digital Singapore Pte Ltd') != -1:
            taxrate = '8.00'
        elif raw_text.find('Next Online Limited') != -1:
            taxrate = '5.00'
        elif raw_text.find('KMC Mag Solutions, Inc.') != -1:
            taxrate = '12.00'
        elif raw_text.find('Jones Lang LaSalle Bangladesh Pvt. Ltd.') != -1:
            taxrate = '15.00'
        elif raw_text.find('Apple Distribution International Ltd.') != -1 or raw_text.find('Aegis BPO Malaysia Sdn Bhd') != -1 or raw_text.find('BLUE WIRELESS PTE. LTD.') != -1 or raw_text.find('Telstra International Limited') != -1 or raw_text.find('Meero SAS') != -1 or raw_text.find('EOS IT MANAGEMENT SOLUTIONS INC') != -1 :
            taxrate = '0.00'

        else:
            taxrate = 'NA'
        return taxrate
    def get_taxamount(raw_text):
        '''Function to extract taxamount from raw_text'''
        if raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd') != -1 and raw_text.find('VAT @')!=-1:
            p=raw_text.find('VAT @')
            taxamount=raw_text[p:p+40].split('\n')[2]
        elif raw_text.find('Equinix Singapore Pte Ltd') != -1 and raw_text.find('GST at 7% on')!=-1:
            p=raw_text.find('GST at 7% on')
            taxamount=raw_text[p:p+80].split('\n')[7]
        elif raw_text.find('SK Solutions') != -1 and raw_text.find('VAT on Service Charge only (15%)')!=-1:
            p=raw_text.find('VAT on Service Charge only (15%)')
            taxamount=raw_text[p:p+150].split('\n')[7]
        elif raw_text.find('S.E. RENTALS PTY LTD')!= -1:
            p=raw_text.find('\nMonthly')
            x=raw_text[p:p+50]
            taxamount=x.split('\n')[3].replace('$','')
        elif raw_text.find('Next Online Limited')!= -1:
            p=raw_text.find('VAT')
            x=raw_text[p:p+50]
            taxamount=x.split('\n')[4]
        elif raw_text.find('Mobitel')!= -1:
            p=raw_text.find('VAT (8.00%)')
            x=raw_text[p:p+100]
            taxamount=x.split('\n')[1]   
        elif raw_text.find('Bespoke Nominees Pty Ltd')!= -1:
            p=raw_text.find('TOTAL GST 10%')
            x=raw_text[p:p+100]
            taxamount=x.split('\n')[1]
        elif raw_text.find('ADA Digital Singapore Pte Ltd')!= -1:
            p=raw_text.find('VAT 8%')
            x=raw_text[p:p+100]
            taxamount=x.split('\n')[1]
        elif raw_text.find('Meero SAS')!= -1 or raw_text.find('BLUE WIRELESS PTE. LTD.')!= -1 or\
        raw_text.find('Apple Distribution International Ltd.') != -1 or raw_text.find('Aegis BPO Malaysia Sdn Bhd')!=-1: 
            taxamount='0.00'
        elif raw_text.find('GroupM Japan K.K.')!= -1 and  raw_text.find('Uber Eats Japan, Inc.')!= -1:
            p=raw_text.find('Subtotal')
            x=raw_text[p:p+150]
            taxamount=x.split('\n')[7]
        elif raw_text.find('MEDIACOM') != -1 and raw_text.find('GroupM Japan K.K.') == -1:
            taxamount = '0.00'
        elif raw_text.find('Quess Corp Lanka Private Limited')!= -1:
            p=raw_text.find('VAT @ 8%')
            x = raw_text[p:p+200]
            taxamount=x.split('\n')[3]
        elif raw_text.find('All Branded Group Pty Limited') != -1:
            p=raw_text.find('Pay by Credit')
            x = raw_text[p-100:p]
            taxamount=x.split('\n')[-4]
        elif raw_text.find('Yakusan Pty Ltd') != -1:
            p=raw_text.find('GST @ 10.000%')
            x = raw_text[p:p+100]
            taxamount=x.split('\n')[1]           
        elif raw_text.find('Hawker Britton Group Pty Ltd') != -1:
            p=raw_text.find('Invoice Total')
            x = raw_text[p:p+100]
            taxamount=x.split('\n')[4]
        elif raw_text.find('INXPRESS AUSTRALIA') != -1:   
            p=raw_text.find('GST Amount')
            x = raw_text[p:p+100]
            taxamount=x.split('\n')[4]
        elif raw_text.find('Yahoo Japan Corporation') != -1:
            p=raw_text.find('Tax Total')
            x=raw_text[p:p+100]
            taxamount=x.split('\n')[1] 
        else:
            taxamount = 'NA'
        return taxamount
    def get_terms(raw_text):
        if raw_text.find('Business Process Outsourcing International') != -1 or \
        raw_text.find('EOS IT MANAGEMENT SOLUTIONS INC')!= -1 or \
        raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd') != -1 or \
        raw_text.find('L&L, Leaven & Co., CPAS')!= -1 or raw_text.find('Detmold New Zealand Sales Ltd')!= -1 or raw_text.find('ADA Digital Singapore Pte Ltd')!= -1 :   
            terms = '30 Days'
        elif raw_text.find('Equinix Singapore Pte Ltd') != -1 :  
            terms='Net 45'
        elif raw_text.find('Meero SAS') != -1: 
            terms='Payment on receipt of invoice'
        elif raw_text.find('Next Online Limited') != -1: 
            terms='Net 15' 
        elif raw_text.find('PowerNET Group Pty Ltd') != -1: 
            terms='Net 14' 
        else:
            terms = 'NA'
        return terms
    def start_end_date_pattern_one(date_format):
        '''pattern = 01 Oct 2021 - 31 Oct 2021'''
        data = date_format.split('-')
        start_date = data[0].strip()
        end_date = data[1].strip()
        
        return (start_date, end_date)

    def start_end_date_pattern_two(date_format):
        '''pattern = October 2021'''
        month_with_30_days_list = ["April", "June", "September", "November", "APRIL", "JUNE", "SEPTEMBER", "NOVEMBER","Apr", "Jun", "Sep", "Nov"]
        data = date_format.split(' ')
        month = data[0].strip()
        year = data[1].strip()
        start_date = '1'+'-'+month+'-'+year
        if month in month_with_30_days_list:
            end_date = "30" + "-" + month + "-" + year
        elif month == "February" or month == "FEBRUARY" or month=='Feb':
            if int(year) % 4 == 0:
                end_date = "29" + "-" + month + "-" + year
            else:
                end_date = "28" + "-" + month + "-" + year
        else:
            end_date = "31" + "-" + month + "-" + year

        return (start_date, end_date)

    def start_end_date_pattern_three(date_format):
        ''' pattern = '5/31/2021'''
        month_with_30_days_list = ["4", "6", "9", "11"]
        data = date_format.split('/')
        month = data[0].strip()
        year = data[2].strip()
        start_date = '1'+'-'+month+'-'+year
        if month in month_with_30_days_list:
            end_date = "30" + "-" + month + "-" + year
        elif month == "February":
            if int(year) % 4 == 0:
                end_date = "29" + "-" + month + "-" + year
            else:
                end_date = "28" + "-" + month + "-" + year
        else:
            end_date = "31" + "-" + month + "-" + year

        return (start_date, end_date)

    def start_end_date_pattern_four(date_format):
        '''pattern = July-Aug 2021'''
        month_with_30_days_list = ["April", "June", "September", "November", "APRIL", "JUNE", "SEPTEMBER", "NOVEMBER","Apr", "Jun", "Sep", "Nov"]
        data = date_format.split(' ')
        month = data[0].strip()
        split_month=month.split('-')
        year = data[1].strip()
        start_date = '1'+'-'+split_month[0]+'-'+year
        if split_month[1] in month_with_30_days_list:
            end_date = "30" + "-" + split_month[1] + "-" + year
        elif split_month[1] == "Feb" or split_month[1] == "FEB" or split_month[1] == "FEBRUARY":
            if int(year) % 4 == 0:
                end_date = "29" + "-" + split_month[1] + "-" + year
            else:
                end_date = "28" + "-" + split_month[1] + "-" + year
        else:
            end_date = "31" + "-" + split_month[1] + "-" + year

        return (start_date, end_date)
    def service_period_start_end_date(data):
        '''Function to extract service start and end date'''
        try:
            if len(data)>20:
                start, end = start_end_date_pattern_one(data)
                return (start, end)
            elif re.fullmatch('\w+\s\d+',data):
                start, end = start_end_date_pattern_two(data)
                return (start, end)
            elif re.fullmatch('\d[/]\d+[/]\d{4}',data):
                start, end = start_end_date_pattern_three(data)
                return (start, end)
            elif re.fullmatch('\w+-\w+\s\d{4}',data):
                start, end = start_end_date_pattern_four(data)
                return (start, end)
            else:
                return('NA', 'NA')
        except:
            return('NA', 'NA')

    def get_service_start_end_date_from_raw_text(raw_text):
        try:
            if raw_text.find('L&L, Leaven & Co., CPAs')!= -1 :
                if raw_text.find('Service Rendered from')!= -1 :
                    search_key="Service Rendered from"
                    sub_string = raw_text[raw_text.find(search_key):raw_text.find(search_key)+50]
                    pattern = re.findall(r'\d{2}.\d{2}.\d{4}', sub_string)
                    start_date, end_date = service_period_start_end_date(pattern[0] +'-' +pattern[1])
                    return (start_date, end_date)
                elif raw_text.find('Fixed Fee')!= -1 :
                    bp_str=raw_text[raw_text.find('Fixed Fee'):raw_text.find('Fixed Fee')+50]
                    pattern = re.findall(r'\w{3,}\s\d{4}', bp_str)
                    start_date, end_date = service_period_start_end_date(pattern[0])
                    return (start_date, end_date)
            elif raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd')!= -1 :
                if raw_text.find('for the month')!= -1 :
                    search_key="for the month"
                    sub_string = raw_text[raw_text.find(search_key):raw_text.find(search_key)+50]
                    sub_string=re.sub("['-]", " ",sub_string)
                    pattern = re.findall('\w{,4}\s\d{2}|\w+\s\d{4}', sub_string)
                    if pattern!=None:
                        start_date, end_date = service_period_start_end_date(pattern[0])
                        return (start_date, end_date)
                    else:
                        return('NA', 'NA')
                else:
                    return('NA', 'NA') 
            elif raw_text.find('Mobitel')!= -1 :
                if raw_text.find('Monthly Bill Summary for the period')!= -1 :
                    start_key = 'Monthly Bill Summary for the period'
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+50]
                    pattern = re.findall('\d+[/]\d{2}', sub_string)
                    start_date, end_date = pattern[0]+'/'+'2021',pattern[1]+'/'+'2021'
                    return (start_date, end_date)
                else:
                    return('NA', 'NA') 
            elif raw_text.find('Business Process Outsourcing International')!= -1 :
                if raw_text.find('PARTICULARS')!= -1 :
                    start_key = 'PARTICULARS'
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+70]
                    pattern = re.findall('\w+\s\d{4}', sub_string)
                    start_date, end_date = service_period_start_end_date(pattern[0])
                    return (start_date, end_date)
                else:
                    return('NA', 'NA') 
            elif raw_text.find('Meero SAS') != -1:
                    start_key ='Invoice from'
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+40]
                    pattern = re.findall('\d+[/]\d+[/]\d{2}', sub_string)
                    start_date, end_date = pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('FUJIFILM Business Innovation Philippines Corp.') != -1:
                    start_key ='DETAILS OF CHARGES FOR'
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+40]
                    pattern = re.findall('\w+\s\d{4}', sub_string)
                    start_date, end_date = service_period_start_end_date(pattern[0])
                    return (start_date, end_date)
            elif raw_text.find('KPMG') != -1:
                    start_key ='service rendered'
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+40]
                    pattern = re.findall('\w+\s\d+|\w+-\w+\s\d{4}', sub_string)
                    start_date, end_date = service_period_start_end_date(pattern[0])
                    return (start_date, end_date)
            elif raw_text.find('S.E. RENTALS PTY LTD') != -1:
                    start_key ='Contract Billing for the Period '
                    sub_string = raw_text[raw_text.find(start_key):raw_text.find(start_key)+100]
                    pattern = re.findall('\d+[/]\d+[/]\d{4}', sub_string)
                    start_date, end_date = pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('Next Online Limited') != -1:
                    p=raw_text.find('BILLING MONTH\n')
                    sub_string = raw_text[p:p+50]
                    sub_string=re.sub(",", "",sub_string)
                    pattern = re.findall('\w+\s\d+', sub_string)
                    start_date, end_date = service_period_start_end_date(pattern[0])
                    return (start_date, end_date)
            elif raw_text.find('Telstra International Limited') != -1:
                    p=raw_text.find('PO#')
                    sub_string = raw_text[p:p+50]
                    pattern=sub_string.split('\n')[1].split('to')
                    start_date, end_date =pattern[0],pattern[1]
                    return (start_date, end_date)
                
            elif raw_text.find('KMC Mag Solutions, Inc.') != -1:
                    p=raw_text.find('For the period of')
                    sub_string = raw_text[p+18:p+40]
                    pattern=re.findall('[A-z]+|\d{4}',sub_string)
                    pattern=' '.join(pattern)
                    start_date, end_date = service_period_start_end_date(pattern)
                    return (start_date, end_date)  
            elif raw_text.find('ADA Digital Singapore Pte Ltd') != -1:
                    p=raw_text.find('Billing Period')
                    q=raw_text.find('Due Date')
                    sub_string = raw_text[p:q]
                    pattern = re.findall('\d+/\d+/\d+', sub_string)
                    if pattern!=None:
                        start_date, end_date = pattern[0],pattern[1]
                        return (start_date, end_date) 
            elif raw_text.find('Dialog Bredbandi betwerks DNA LEN Ý') != -1:
                    p=raw_text.find('BILL PERIOD')
                    sub_string= raw_text[p-50:p+50]
                    pattern = re.findall('\w+\s\d+-\d+,\s\d+|\d+/\d+/\d+', sub_string)
                    if pattern!=None:
                        start_date, end_date = pattern[0],pattern[1]
                        return (start_date, end_date)
            elif raw_text.find('Aon Singapore Pte. Ltd.') != -1:
                    p=raw_text.find('Policy Period')
                    sub_string = raw_text[p:p+100]
                    pattern=sub_string.split('\n')[1].split('to')
                    start_date, end_date = pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('BLUE WIRELESS PTE. LTD.') != -1: 
                    p=raw_text.find('Active Month')
                    x = raw_text[p:p+70]
                    desc_value=x.split('\n')[0].replace('-',' ')
                    pattern = re.findall('\w+\s\d+', desc_value)
                    if pattern!=None:
                        start_date, end_date =service_period_start_end_date(pattern[0])
                        return (start_date, end_date)                  
            elif raw_text.find('Yahoo Japan Corporation') != -1:      
                    p=raw_text.find('Search Ads ad spend for the month')
                    x=raw_text[p:p+100]
                    pattern=x.split('\n')[1].split('-')
                    start_date, end_date=pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('PCCWⓇ') != -1:      
                    p=raw_text.find('Bill Period')
                    x=raw_text[p:p+100]
                    pattern=x.split('\n')[1].split('to')
                    start_date, end_date=pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('Apple Distribution International Ltd.') != -1:    
                    p=raw_text.find('Billing Period:')
                    x=raw_text[p:p+100]
                    pattern=x.split('\n')[1].split('-') 
                    start_date, end_date=pattern[0],pattern[1]
                    return (start_date, end_date)
            elif raw_text.find('Aegis BPO Malaysia Sdn Bhd') != -1:                    
                    p=raw_text.find('Monthly Recurring Fee')
                    x = raw_text[p:p+70]
                    desc_value=x.split('\n')[0].replace('-',' ')
                    pattern = re.findall('\w+\s\d+', desc_value)
                    if pattern!=None:
                        start_date, end_date =service_period_start_end_date(pattern[0])
                        return (start_date, end_date) 
            elif raw_text.find('70503981') != -1:                    
                    p=raw_text.find('*HẸVÃO')
                    x = raw_text[p:p+100]
                    desc_value=x.split('\n')[0].split(' ')[1]
                    start_date, end_date =y.split('~')[0],y.split('~')[1]
                    return (start_date, end_date)        

               
            else:
                return('NA', 'NA')
        except:
            return ('NA','NA')

    


    #Iterating over forms_data  PARTICULARS
    for form in forms_data:
        
        #PO
        if form['fieldName'].startswith(tuple(po_list)):
            if PO_NUMBER_OUT_KEY_FLAG:
                value = form['fieldValue'].strip()
               # print(form['fieldName'],value)
                if value.strip() != '':
                    parsed_data[PO_NUMBER_OUT_KEY]['text'] =''.join(re.findall(r'\d{6}',value))
                    parsed_data[PO_NUMBER_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[PO_NUMBER_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[PO_NUMBER_OUT_KEY]['page_idx'] = form['page_idx']
                    PO_NUMBER_OUT_KEY_FLAG = False
                    
        #Invoice_date       
        if form['fieldName'].startswith(tuple(invoice_date_list)):
            if INV_DATE_OUT_KEY_FLAG:
                value = form['fieldValue'].strip()
                if value.strip() != '':
                    parsed_data[INV_DATE_OUT_KEY]['text'] = value.replace(':','')
                    parsed_data[INV_DATE_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[INV_DATE_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[INV_DATE_OUT_KEY]['page_idx'] = form['page_idx']
                    INV_DATE_OUT_KEY_FLAG = False

                    
                    
        #Invoice_number  
        if form['fieldName'].startswith(tuple(invoice_no_list)):
            if INV_NO_OUT_KEY_FLAG:
                value = form['fieldValue'].strip()
                if value.strip() != '':
                    parsed_data[INV_NO_OUT_KEY]['text'] =  value.replace(':','')
                    parsed_data[INV_NO_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[INV_NO_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[INV_NO_OUT_KEY]['page_idx'] = form['page_idx']
                    INV_NO_OUT_KEY_FLAG = False
        
        #subtotal  
        if  form['fieldName'].strip() in tuple(subtotal_list):
            if SUB_TOTAL_OUT_KEY_FLAG:            
                value = form['fieldValue'].strip()
                #print(form['fieldName'],value)
                if value.strip() != '':
                    parsed_data[SUB_TOTAL_OUT_KEY]['text'] = re.sub('[A-z]*\n|[A-z](.*?)|[$](.*?)|^0.00', '', value)
                    parsed_data[SUB_TOTAL_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[SUB_TOTAL_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[SUB_TOTAL_OUT_KEY]['page_idx'] = form['page_idx']
                    SUB_TOTAL_OUT_KEY_FLAG = False
        
         #Invoice_Amount 
        if  form['fieldName'].strip() in tuple(invoice_amount_list):
            if TOTAL_OUT_KEY_FLAG:
                value = form['fieldValue'].strip()
                
                if value.strip() != '':
                    parsed_data[TOTAL_OUT_KEY]['text'] = re.sub('[A-z]*\n|[A-z](.*?)|[$](.*?)|^0.00|\s', '', value)
                    parsed_data[TOTAL_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[TOTAL_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[TOTAL_OUT_KEY]['page_idx'] = form['page_idx']
                    TOTAL_OUT_KEY_FLAG = False
        #TAX Amount 
        if form['fieldName'].strip() in tuple(tax_amount_list):
            if TAX_OUT_KEY_FLAG:
                value = form['fieldValue'].strip()
                
                if value.strip() != '':
                    parsed_data[TAX_OUT_KEY]['text'] = re.sub('[A-z]*\n|[A-z](.*?)|[$](.*?)', '', value)
                    parsed_data[TAX_OUT_KEY]['conf_score'] = form['nameConfidence']
                    parsed_data[TAX_OUT_KEY]['coordinates'] = form['nameCoordinates']
                    parsed_data[TAX_OUT_KEY]['page_idx'] = form['page_idx']
                    TAX_OUT_KEY_FLAG = False
            #extracting supplier and legal_entity
        if SUPPLIER_OUT_KEY_FLAG:
            supplier, legal_entity = get_supplier_legal_entity(raw_text)
            if supplier != 'NA':
                parsed_data[SUPPLIER_OUT_KEY]['text'] = supplier
                parsed_data[SUPPLIER_OUT_KEY]['page_idx'] = 1
            if legal_entity != 'NA':
                parsed_data[ENTITY_OUT_KEY]['text'] = legal_entity
                parsed_data[ENTITY_OUT_KEY]['page_idx'] = 1
                SUPPLIER_OUT_KEY_FLAG = False
        
    
    #extracting Currency
        if CURRENCY_OUT_KEY_FLAG:
            currency = get_currency(raw_text)
            if currency != 'NA':
                parsed_data[CURRENCY_OUT_KEY]['text'] = currency
                parsed_data[CURRENCY_OUT_KEY]['page_idx'] = 1
                CURRENCY_OUT_KEY_FLAG = False
        if PO_NUMBER_OUT_KEY_FLAG:
            po_no=get_po_no(raw_text)
            if po_no!='NA':
                parsed_data[PO_NUMBER_OUT_KEY]['text'] =''.join(re.findall(r'\d{6}',po_no))
                parsed_data[PO_NUMBER_OUT_KEY]['page_idx'] = 1
                PO_NUMBER_OUT_KEY_FLAG=False

        if INV_DATE_OUT_KEY_FLAG:
            invoice_date=get_invoice_date(raw_text)
            if invoice_date != 'NA':
                parsed_data[INV_DATE_OUT_KEY]['text'] = invoice_date
                parsed_data[INV_DATE_OUT_KEY]['page_idx'] = 1
                INV_DATE_OUT_KEY_FLAG = False
        if INV_NO_OUT_KEY_FLAG:
            invoice_no=get_invoice_no(raw_text)
            if invoice_no != 'NA':
                parsed_data[INV_NO_OUT_KEY]['text'] = invoice_no
                parsed_data[INV_NO_OUT_KEY]['page_idx'] = 1
                INV_NO_OUT_KEY_FLAG = False
        if SUB_TOTAL_OUT_KEY_FLAG:
            subtotal=get_subtotal(raw_text)
            if subtotal != 'NA':
                parsed_data[SUB_TOTAL_OUT_KEY]['text'] = subtotal
                parsed_data[SUB_TOTAL_OUT_KEY]['page_idx'] = 1
                SUB_TOTAL_OUT_KEY_FLAG = False

        if TOTAL_OUT_KEY_FLAG:
            invoice_amount=get_invoice_amount(raw_text)
            if invoice_amount != 'NA':
                parsed_data[TOTAL_OUT_KEY]['text'] = invoice_amount
                parsed_data[TOTAL_OUT_KEY]['page_idx'] = 1
                TOTAL_OUT_KEY_FLAG = False 
            #extracting Taxrate
        if TAX_RATE_OUT_KEY_FLAG:
            taxrate = get_taxrate(raw_text)
            if taxrate != 'NA':
                parsed_data[TAX_RATE_OUT_KEY]['text'] = taxrate
                parsed_data[TAX_RATE_OUT_KEY]['page_idx'] = 1
                TAX_RATE_OUT_KEY_FLAG = False
        if TAX_OUT_KEY_FLAG:
            taxamount = get_taxamount(raw_text)
            if taxamount != 'NA':
                parsed_data[TAX_OUT_KEY]['text'] = taxamount
                parsed_data[TAX_OUT_KEY]['page_idx'] = 1
                TAX_OUT_KEY_FLAG = False

        if TERMS_OUT_KEY_FLAG:
            terms = get_terms(raw_text)
            if terms != 'NA':
                parsed_data[TERMS_OUT_KEY]['text'] = terms
                parsed_data[TERMS_OUT_KEY]['page_idx'] = 1
                TERMS_OUT_KEY_FLAG = False
        if SERVICE_START_DATE_OUT_KEY_FLAG or SERVICE_END_DATE_OUT_KEY_FLAG :
            start_date,end_date = get_service_start_end_date_from_raw_text(raw_text)
            if start_date != 'NA':
                parsed_data[SERVICE_START_DATE_OUT_KEY]['text'] = start_date
                parsed_data[SERVICE_START_DATE_OUT_KEY]['page_idx'] = 1
                SERVICE_START_DATE_OUT_KEY_FLAG = False
            if end_date != 'NA':
                parsed_data[SERVICE_END_DATE_OUT_KEY]['text'] = end_date
                parsed_data[SERVICE_END_DATE_OUT_KEY]['page_idx'] = 1
                SERVICE_END_DATE_OUT_KEY_FLAG = False
            
        

    return[200, parsed_data]


In [145]:
def table_json(data):
    # KEYS
    DESCRIPTION_KEY = 'Description'
    QTY = 'Qty_Invoiced'
    UNIT_PRICE = 'Unit_Price'
    TAX_AMOUNT = 'Tax_Amount'
    TAX_RATE = 'Tax_Rate'
    LINE_TOTAL_AMOUNT_KEY = 'Line_Total/Amount'

    parsed_table_data = {}
    keys = [DESCRIPTION_KEY, QTY, UNIT_PRICE, TAX_AMOUNT, TAX_RATE, LINE_TOTAL_AMOUNT_KEY]
    
    #data = json.loads(data)
    raw_text, pages = data['raw_text'], data['pages']
    forms_data = pages[0]['forms_data']
    pages_dimension = pages[0]["dimension"]
    tables_data = []
    for page in range(len(pages)):
        tables_data.extend(pages[page]['tables_data'])
    
    tables = [table_data['table'] for table_data in tables_data]
    table_no = 1
    table_data = []
    #############table data from raw text start##########
    if raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Uber - Mawatha')!=-1:
            p=raw_text.find('Uber - Mawatha')
            x=raw_text[p:p+300]
            
            desc_value=x.split('\n')[1]
            line_value=x.split('\n')[2]
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1 
            table_data.append(table_json)
    if raw_text.find('L&L, Leaven & Co.')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Service Fees')!=-1:
            p=raw_text.find('Service Fees')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[0]
            line_value=x.split('\n')[3]
        elif raw_text.find('Fixed Fee')!=-1:
            p=raw_text.find('Fixed Fee')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[0]
            line_value=x.split('\n')[2]
        table_json['Description']['text'] = desc_value
        table_json['Description']['page_idx'] = 1
        table_json['Line_Total/Amount']['text'] = line_value
        table_json['Line_Total/Amount']['page_idx'] = 1 
        table_data.append(table_json)
    if raw_text.find('Business Process Outsourcing International')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('PARTICULARS')!=-1:
            p=raw_text.find('PARTICULARS')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[2]
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_data.append(table_json)
    if raw_text.find('PowerNET Group Pty Ltd')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Managed Services Details')!=-1:
            p=raw_text.find('Managed Services Details')
            x=raw_text[p:p+150]
            desc_value=x.split('\n')[1]
            qty_value=x.split('\n')[2]
            unit_value=x.split('\n')[3]
            line_value=x.split('\n')[4]
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Qty_Invoiced']['text'] =qty_value
            table_json['Qty_Invoiced']['page_idx'] = 1
            table_json['Unit_Price']['text'] = unit_value
            table_json['Unit_Price']['page_idx'] = 1 
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1 
            table_data.append(table_json)
    if raw_text.find('Bespoke Nominees Pty Ltd')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Subtotal')!=-1:
            p=raw_text.find('Subtotal')
            x=raw_text[p:p+100]
            desc_value='Professional Services'
            line_value=x.split('\n')[1]
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1 
            table_data.append(table_json)
    if raw_text.find('Quess Corp Lanka Private Limited')!=-1:  
        #table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Particulars')!=-1:
            p=raw_text.find('Particulars')
            x=raw_text[p:p+200]
            desc_value=x.split('\n')[1:3]
            line_value=x.split('\n')[4:6]
            for i,(des,line) in enumerate(zip(desc_value,line_value)):
                table_json = get_default_format(keys, pages_dimension)
                table_json['Description']['text'] = des
                table_json['Description']['page_idx'] = 1
                table_json['Line_Total/Amount']['text'] = line
                table_json['Line_Total/Amount']['page_idx'] = 1 
                table_data.append(table_json)
    if raw_text.find('Special Group Australia Pty Ltd')!=-1:  
        #table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('ABN')!=-1:
            p=raw_text.find('ABN')
            x=raw_text[p:p+200]
            desc_value=x.split('\n')[1:4]
            line_value=x.split('\n')[4:7]
            for i,(des,line) in enumerate(zip(desc_value,line_value)):
                table_json = get_default_format(keys, pages_dimension)
                table_json['Description']['text'] = des
                table_json['Description']['page_idx'] = 1
                table_json['Line_Total/Amount']['text'] = line
                table_json['Line_Total/Amount']['page_idx'] = 1 
                table_data.append(table_json)         
    if raw_text.find('Superior Air Conditioning')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('TOTAL PRICE')!=-1:
            p=raw_text.find('TOTAL PRICE')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[1]
            line_value=x.split('\n')[2]
            
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1
            table_data.append(table_json)
    if raw_text.find('Yakusan Pty Ltd')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('Client Order No:')!=-1:
            p=raw_text.find('Client Order No:')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[3]
            line_value=x.split('\n')[4]
            
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1
            table_data.append(table_json)
    if raw_text.find('First Aurora Limited (HK)')!=-1:  
        table_json = get_default_format(keys, pages_dimension)
        if raw_text.find('DESCRIPTION')!=-1:
            p=raw_text.find('DESCRIPTION')
            x=raw_text[p:p+100]
            desc_value=x.split('\n')[7]
            qty_value=x.split('\n')[4]
            unit_value=x.split('\n')[5]
            line_value=x.split('\n')[6]
            
            table_json['Description']['text'] = desc_value
            table_json['Description']['page_idx'] = 1
            table_json['Qty_Invoiced']['text'] =qty_value
            table_json['Qty_Invoiced']['page_idx'] = 1
            table_json['Unit_Price']['text'] = unit_value
            table_json['Unit_Price']['page_idx'] = 1 
            table_json['Line_Total/Amount']['text'] = line_value
            table_json['Line_Total/Amount']['page_idx'] = 1
            table_data.append(table_json) 

    #############table data from raw text end##########
    try:
        for table_enu,table in enumerate(tables):
            for column_enu,columns in enumerate(table):
                check_row = ''.join(columns) 
                
                if check_row.find('Description') != -1 and check_row.find('Quantity') != -1 and check_row.find('Tax') == -1:
                    table_no = table_enu
                    break
                elif check_row.find('PRODUCT DESCRIPTION') != -1 and check_row.find('NET AMOUNT') != -1 and check_row.find('PRICE') != -1 and check_row.find('QUANTITY') != -1:
                    table_no = table_enu                    
                    break
                elif check_row.find('Item Description') != -1 and check_row.find('Cartons') != -1:
                    table_no = table_enu
                    break
                elif check_row.find('ACTIVITY') != -1 and check_row.find('AMOUNT') != -1:
                    table_no = table_enu
                    break
                elif check_row.find('Asset Description') != -1 and check_row.find('Net') != -1:
                    table_no = table_enu
                    break
                elif check_row.find('Placement') != -1 and check_row.find('Qty') != -1:
                    table_no = table_enu  
                    break
                elif check_row.find('Charge Description') != -1 and check_row.find('Unit Price') != -1:
                    table_no = table_enu  
                    break
                elif check_row.find('Descriptions') != -1 and check_row.find('Qnty') != -1:
                    table_no = table_enu                      
                    break
                elif check_row.find('Descriptions') != -1 and check_row.find('Rate') != -1:
                    table_no = table_enu                      
                    break
                elif check_row.find('Product Description') != -1 and check_row.find('Qty') != -1:
                    table_no = table_enu  
                    break
                elif check_row.find('Start(A)') != -1 and check_row.find('End (B)') != -1:
                    table_no = table_enu  
                    break
                elif check_row.find('order details') != -1 and check_row.find('unit price') != -1:
                    table_no = table_enu                      
                    break
                elif check_row.find('Media') != -1 and check_row.find('Commission') != -1:
                    table_no = table_enu                      
                    break 
                elif check_row.find('Monthly Subscriptions') != -1 or check_row.find('Call Charges (Voice/Video)') != -1:
                    table_no = table_enu                      
                    break
                elif check_row.find('Voice') != -1 or check_row.find('Data') != -1:
                    table_no = table_enu
                    break
                elif check_row.find('Description') != -1 and check_row.find('Tax') != -1 and check_row.find('Amount PHP') != -1 and check_row.find('Quantity') != -1:
                    table_no = table_enu                    
                    break
                elif check_row.find('Description') != -1 and check_row.find('Amount in BDT') != -1:
                    table_no = table_enu                   
                    break
                elif check_row.find('Description') != -1 and check_row.find('Amoun') != -1:
                    table_no = table_enu                   
                    break
                elif check_row.find('Subject') != -1 and check_row.find('Number of unit') != -1  and check_row.find('Unit price') != -1:
                    table_no = table_enu                   
                    break
                  
                elif check_row.find('DESCRIPTIONS') != -1 and check_row.find('QTY') != -1  and check_row.find('UNIT PRICE') != -1:
                    table_no = table_enu 
                    break
                elif check_row.find('Hire and Maintenance of Interior Plantscapes') != -1 :
                    table_no = table_enu 
                    break
                elif check_row.find('Item Code Name')!= -1 and check_row.find('Unit Price')!=-1  and check_row.find('Price') != -1 :
                    table_no = table_enu 
                    break
                elif check_row.find('Items')!= -1 and check_row.find('Amount')!=-1 and check_row.find('Quantity')!=-1  and check_row.find('Price') != -1 :
                    table_no = table_enu 
                    break
                elif check_row.find('Description') != -1 and check_row.find('Total') != -1 and check_row.find('Tax') != -1:
                    table_no = table_enu 
                   
                    break
                elif check_row.find('Item') != -1 and check_row.find('Quantity') != -1 and check_row.find('Unit Price') != -1 and check_row.find('Amount') != -1:
                    table_no = table_enu                   
                    break
                elif check_row.find('ITEM') != -1 and check_row.find('QTY') != -1 and check_row.find('PRICE\nAMOUNT') != -1:
                    table_no = table_enu                   
                    break                
                elif check_row.find('Item Description') != -1 and check_row.find('Nett Price') != -1 and check_row.find('Line Total') != -1:
                    table_no = table_enu                   
                    break
                elif check_row.find('Description') != -1 and check_row.find('Tax Rate') != -1 and check_row.find('Terms') != -1:
                    table_no = table_enu 
                    break
                elif check_row.find('Job Number') != -1 :
                    table_no = table_enu 
                    break
                elif check_row.find('Description') != -1 and check_row.find('Qty') != -1 and check_row.find('Price') != -1 and check_row.find('Cost') != -1:
                    table_no = table_enu                   
                    break
                elif check_row.find('Description') != -1 and check_row.find('Qty-Billable') != -1 and check_row.find('Amount') != -1:
                    table_no = table_enu                   
                    break


        table = tables[table_no]
       
        for rows in table:
            if   raw_text.find('First Aurora Limited (HK)')!=-1 or raw_text.find('Yakusan Pty Ltd')!=-1 or raw_text.find('Superior Air Conditioning')!=-1 or raw_text.find('Special Group Australia Pty Ltd')!=-1  or raw_text.find('Quess Corp Lanka Private Limited')!=-1  or raw_text.find('Bespoke Nominees Pty Ltd')!=-1 or raw_text.find('PowerNET Group Pty Ltd')!=-1 or raw_text.find('Jones Lang Lasalle Lanka (Private) Ltd')!=-1 or raw_text.find('Business Process Outsourcing International')!=-1  or raw_text.find('L&L, Leaven & Co.')!=-1: 
                break
            table_json = get_default_format(keys, pages_dimension)
            #ada digital  #leaf
            if ''.join(rows).find("Description") != -1 and ''.join(rows).find("Quantity") != -1 and ''.join(rows).find("Price") != -1 and ''.join(rows).find("Amount PHP") == -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Quantity', item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Price', item)]))
                    index_line_amount=-1                   
                except:
                    pass
                    continue
            #cdw it austrelia
            elif ''.join(rows).find("PRODUCT DESCRIPTION") != -1 and ''.join(rows).find("QUANTITY") != -1 and ''.join(rows).find('GST %') == -1 and ''.join(rows).find('PRICE') != -1 :                
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRODUCT DESCRIPTION', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('QUANTITY', item)]))
                    index_unit_price=idx=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRICE', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('NET AMOUNT', item)]))  
                except:
                    pass
                    continue
            #cdw it singapur
            elif ''.join(rows).find("PRODUCT DESCRIPTION") != -1 and ''.join(rows).find("QUANTITY") != -1 and ''.join(rows).find('GST %') != -1 :                
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRODUCT DESCRIPTION', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('QUANTITY', item)]))
                    index_unit_price=idx=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRICE', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('NET AMOUNT', item)]))
                    index_tax_rate=int(''.join([str(i) for i, item in enumerate(rows) if re.search('GST %', item)]))                
                except:
                    pass
                    continue
            #detmold
            elif ''.join(rows).find("Item Description") != -1 and ''.join(rows).find("Cartons") != -1 :
                try:
                    index_des=1
                    index_qnty=rows.index('Cartons\nShipped\n')                    
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Total', item)]))                    
                except:
                    pass
                    continue
            #next online
            elif ''.join(rows).find("ACTIVITY") != -1 and ''.join(rows).find("AMOUNT") != -1 :                
                index_des=rows.index('ACTIVITY\n')
                index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('QTY', item)]))
                index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('RATE', item)]))
                index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('AMOUNT', item)]))                
                continue
            #se rentals
            elif ''.join(rows).find("Asset Description") != -1 and ''.join(rows).find("Net") != -1 :                
                index_des=rows.index('Asset Description\n')                
                index_gst_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('GST', item)]))
                index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Net', item)]))                
                continue
            #kmc mag
            elif ''.join(rows).find("Item Description") != -1 and ''.join(rows).find("Quantity") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Quantity', item)]))                    
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Total Cost', item)]))                    
                except:
                    pass
                    continue
            #mediacom
            elif ''.join(rows).find("Placement") != -1 and ''.join(rows).find("Qty") != -1 :               
                index_des=rows.index('Placement\n')
                index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Qty', item)]))               
                index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Total', item)]))               
                continue
            #telestra
            elif ''.join(rows).find("Charge Description") != -1 and ''.join(rows).find("Unit Price") != -1 :                
                try:
                    index_des=rows.index('Charge Description\n')                    
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit Price', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Total', item)]))                    
                except:
                    pass
                    continue 
            #sk solutions
            elif ''.join(rows).find("Descriptions") != -1 and (''.join(rows).find("Qnty") != -1 or ''.join(rows).find("SFT") != -1):                
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Descriptions', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('(Qnty|SFT)', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('(Tk.|TK.)',item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit Price|Rate|CPU', item)]))                    
                except:
                    pass
                    continue
            #eos it
            elif ''.join(rows).find("Product Description") != -1 and ''.join(rows).find("Qty") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Product Description', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Qty', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Net Price', item)]))
                except:
                    pass
                    continue
            #fuji
            elif ''.join(rows).find("Start") != -1 and ''.join(rows).find("End (B)") != -1 :                
                try:
                    index_des=0
                    index_line_amount=-1
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Net Copies', item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Price Per', item)]))
                except:
                    pass
                    continue
            #meero sas
            elif ''.join(rows).find("order details") != -1 and ''.join(rows).find("unit price") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('order details', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('quantity', item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('price taxes excluded', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('price taxes excluded', item)]))                
                except:
                    pass
                    continue
            #groupm
            elif ''.join(rows).find("Media") != -1 and ''.join(rows).find("Commission") != -1 :
                try:
                    index_des=0
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Commission', item)]))
                except:
                    pass
                    continue
            #giftway
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Quantity") != -1  and ''.join(rows).find("Amount PHP") != -1  and ''.join(rows).find("Tax") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Quantity', item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit Price', item)]))
                    index_tax_rate=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Tax', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Amount PHP',item)]))
                   
                except:
                    pass
                    continue             
            #jones bangladesh
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Amount in BDT") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('in BDT', item)]))
                except:
                    pass
                    continue  
            #adplanet communication
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Amoun") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_line_amount=-1
                except:
                    pass
                    continue 
            #EPI ET RIZ K.K.
            elif ''.join(rows).find("Subject") != -1 and ''.join(rows).find("Unit price") != -1  and ''.join(rows).find("Number of unit") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Subject', item)]))
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Number of unit', item)]))
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit price', item)]))
                    index_line_amount=-1
                except:
                    pass
                    continue  
            #HAKUHODO
            elif ''.join(rows).find("DESCRIPTIONS") != -1 and ''.join(rows).find("QTY") != -1  and ''.join(rows).find("UNIT PRICE") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('DESCRIPTIONS', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('QTY', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('UNIT PRICE', item)]))
                   
                    index_line_amount=-1
                   
                except:
                    pass
                    continue   
            #Carripook Interior Plant Care
            elif ''.join(rows).find("Hire and Maintenance of Interior") != -1:
                try:
                   
                    index_des=0
                    index_line_amount=-1
                except:
                    pass
                    continue 
            #Computec Engineering, Ltd
            elif ''.join(rows).find("Item Code Name") != -1 and ''.join(rows).find("Unit Price") != -1 and ''.join(rows).find("Price") != -1:
                try:
                   
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Item Code Name', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Qty', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit Price', item)]))
                   
                    index_line_amount=4
                   
                except:
                    pass
                    continue  
            #AMBAGAHA DEPELE GEDARA JAGATH KELUM DISSANAYAKE
            elif ''.join(rows).find("Items") != -1 and ''.join(rows).find("Quantity") != -1 and ''.join(rows).find("Amount") != -1 and ''.join(rows).find("Price") != -1:
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Items', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Quantity', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Price', item)]))
                   
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Amount', item)]))
                   
                except:
                    pass
                    continue  
            #HH Australia Pty Ltd
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Total") != -1 and ''.join(rows).find("Tax") != -1 :
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                    index_tax_rate=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Tax', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Total', item)]))
                except:
                    pass
                    continue 
                
            #Kavindu Sankalpa Gunawardana
            elif ''.join(rows).find("Item") != -1 and ''.join(rows).find("Quantity") != -1 and ''.join(rows).find("Amount") != -1 and ''.join(rows).find("Unit Price") != -1:
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Item', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Quantity', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Unit Price', item)]))
                   
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Amount', item)]))
                   
                except:
                    pass
                    continue 
                
            #All Branded Group Pty Limited
            elif ''.join(rows).find("ITEM") != -1 and ''.join(rows).find("QTY") != -1 and ''.join(rows).find("PRICE\nAMOUNT") != -1:
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('ITEM', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('QTY', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRICE\nAMOUNT', item)]))
                   
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('PRICE\nAMOUNT', item)]))
                   
                except:
                    pass
                    continue  
                
            #1043974-CONSOLIDATED OFFICE CHOICE
            elif ''.join(rows).find("Item Description") != -1 and ''.join(rows).find("Nett Price") != -1 and ''.join(rows).find("GST") != -1:
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Item Description', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Ordered', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Nett Price', item)]))
                   
                    index_gst_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('GST ', item)]))
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Line Total', item)]))
                   
                except:
                    pass
                    continue 
            #Yahoo Japan Corporation
            elif ''.join(rows).find("Descripion") != -1 and ''.join(rows).find("Terms") != -1 and ''.join(rows).find("Tax Rate") != -1:
                try:
                    
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Descripion', item)]))
                    
                    index_tax_rate=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Tax Rate', item)]))
                    index_gst_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Tax Rate', item)]))
                    index_line_amount=-1
                    
                except:
                    pass
                    continue 
            #60764-HAWKER BRITTON GROUP PTY LTD
            elif ''.join(rows).find("Job Number") != -1 :
                try:
                    
                    index_des=0
                    index_line_amount=-1
                    
                except:
                    pass
                    continue 
            #1049777-MTC KK
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Qty") != -1 and ''.join(rows).find("Cost") != -1:
                try:
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Qty', item)]))
                   
                    index_unit_price=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Cost', item)]))
                   
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Price', item)]))
                   
                except:
                    pass
                    continue
            #77629 - AEGIS BPO MALAYSIA SDN BHD
            elif ''.join(rows).find("Description") != -1 and ''.join(rows).find("Qty-Billable") != -1 and ''.join(rows).find("Amount") != -1:
               
                try:
                   
                    index_des=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Description', item)]))
                   
                    index_qnty=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Qty-Billable\nHrs/Headcount', item)]))
                   
                    index_line_amount=int(''.join([str(i) for i, item in enumerate(rows) if re.search('Amount', item)]))
                   
                except:
                    pass
                    continue

            elif rows.count('')>=3 and len(rows)<=5:
                
                pass
            elif rows[0]=='' and  rows[-1]=='':
                
                pass
            elif rows[0]=='' and  rows[1]=='':
                
                pass
            elif ''.join(rows).find('In words')!=-1:
                
                pass
            else:
                
                if raw_text.find('SK SOLUTIONS')!=-1:
                    
                    if ''.join(rows).find('Client')!=-1 or ''.join(rows).find('PO Date')!=-1:
                        continue 
                elif raw_text.find('ADA Digital Singapore Pte Ltd')!=-1 or raw_text.find('EOS IT MANAGEMENT SOLUTIONS INC')!=-1 or raw_text.find('GroupM Japan K.K.')!=-1 or raw_text.find('Detmold New Zealand Sales Ltd')!=-1:
                    if ''.join(rows).find('Total')!=-1:
                        continue 
                elif raw_text.find('CDW IT Solutions')!=-1:
                    if ''.join(rows).find('AMOUNT')!=-1 or ''.join(rows).find('GST RATE')!=-1  or ''.join(rows).find('NET')!=-1 or ''.join(rows).find('ACCOUNT NO')!=-1:
                        continue 
                #mobitel
                elif ''.join(rows).find("Monthly Subscriptions") != -1 or ''.join(rows).find("Call Charges") != -1:
                    index_des=0
                    index_line_amount=-1
                elif ''.join(rows).find("Total Amount Payable") != -1 or ''.join(rows).find("Total Payable") != -1 or ''.join(rows).find("VAT") != -1 or ''.join(rows).find("Total Charge") != -1 or ''.join(rows).find("Arrears") != -1:
                    continue
                #Dialouge
                elif ''.join(rows).find("Voice") != -1 or ''.join(rows).find("Data") != -1 :
                    index_des=0
                    index_line_amount=-1  
                elif  ''.join(rows).find('*SGD Equivalent Conversion') != -1 or ''.join(rows).find('Total') != -1 or ''.join(rows).find('Tax') != -1 or ''.join(rows).find('would') != -1 or ''.join(rows).find('Sub Total') != -1 or ''.join(rows).find('Paid') != -1 or ''.join(rows).find('PLEASE NOTE') != -1 or ''.join(rows).find('GST') != -1 or ''.join(rows).find('Voubtotal') != -1 or  ''.join(rows).find("TOTAL") != -1 :
                    continue
                                                                                                                        
                #description
                try:
                    desc_value = rows[index_des].replace('\n', ' ').strip() 
                    table_json['Description']['text'] = desc_value
                    table_json['Description']['page_idx'] = 1
                except:
                    pass
                try:
                    #qty_value = rows[index_qnty].replace('\n', ' ').strip()
                    qty_value = rows[index_qnty].split('\n')[0]
                    if len(qty_value)<4:
                        table_json['Qty_Invoiced']['text'] =qty_value
                        table_json['Qty_Invoiced']['page_idx'] = 1
                except:
                    pass
                try:
                    #unit_value =rows[index_unit_price].replace('\n', '').strip()[A-z]*\n
                    unit_value =re.sub('[A-z](.*?)|[$](.*?)','',rows[index_unit_price] ) 
                    unit_value=unit_value.split('\n')[0]
                    table_json['Unit_Price']['text'] = unit_value
                    table_json['Unit_Price']['page_idx'] = 1                       
                except:
                    pass  
                #Line_total/Subtotal
                try:
                    
                    line_value =re.sub('[A-z](.*?)|[$](.*?)','',rows[index_line_amount] )
                    line_value =line_value.strip()
                    if line_value.count('\n')>=1:
                        line_value=line_value.split('\n')[1]
                    table_json['Line_Total/Amount']['text'] = line_value
                    table_json['Line_Total/Amount']['page_idx'] = 1 
    
                except:
                    pass
                try:
                    Tax_rate =re.sub('[$](.*?)','',rows[index_tax_rate] ) 
                    Tax_rate =Tax_rate.strip()
                    if Tax_rate.count('\n')>=1:
                        Tax_rate=Tax_rate.split('\n')[0]
                    table_json['Tax_Rate']['text'] = Tax_rate
                    table_json['Tax_Rate']['page_idx'] = 1
                except:
                    pass   
                try:
                    Tax_Amount =re.sub('[A-z]*\n|[A-z](.*?)|[$](.*?)','',rows[index_gst_amount] )   
                    table_json['Tax_Amount']['text'] = Tax_Amount
                    table_json['Tax_Amount']['page_idx'] = 1
                except:
                    pass                                  
                table_data.append(table_json)
        parsed_table_data['table1'] = {}
        parsed_table_data['table1']['text'] = pd.DataFrame(table_data).to_json(orient="records")
        parsed_table_data['table1']['conf_score'] = tables_data[table_no]['confidence']
        parsed_table_data['table1']['coordinates'] = tables_data[table_no]['coordinates']
        parsed_table_data['table1']['page_idx'] = tables_data[table_no]['page_idx'] 

        return[200, parsed_table_data]
    except:
        pass


In [146]:
df = pd.DataFrame(dataset, columns=['class_folder_path', 'raw_json', 'class_pred'])
extract_table =[]
for file,data in df['raw_json'].items():
    b = table_json(data)
    print(b)
    print('**********************************')
    
    extract_table.append(b)

[200, {'table1': {'text': '[{"Description":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"},"Qty_Invoiced":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"},"Unit_Price":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"},"Tax_Amount":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"},"Tax_Rate":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"},"Line_Total\\/Amount":{"text":"NA","conf_score":"NA","coordinates":[{"x":0,"y":0},{"x":1758.0,"y":0},{"x":1758.0,"y":2275.0},{"x":0,"y":2275.0}],"page_idx":"NA"}},{"Description":{"text":"NA","conf_score":"

In [147]:
for i in range(len(extract_table)):
    json2 = {}   
    print(i)
    try:
        list1 = json.loads(extract_table[i][1]['table1']['text'])
        rows = []
        for element in list1:
            json2 = {}
            for key,value in element.items():
                json2[key] = value['text']
            rows.append(json2)
        display(pd.DataFrame(rows))
    except:
        print(extract_table[i])

0


,Description,Qty_Invoiced,Unit_Price,Tax_Amount,Tax_Rate,Line_Total/Amount
0,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA


1


,Description,Qty_Invoiced,Unit_Price,Tax_Amount,Tax_Rate,Line_Total/Amount
0,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA
2,NA,NA,NA,NA,NA,NA
3,NA,NA,NA,NA,NA,NA
4,NA,NA,NA,NA,NA,NA


In [148]:
df = pd.DataFrame(dataset, columns=['class_folder_path', 'raw_json', 'class_pred'])
def extract_data(dictionary):
    parsed_data = entities(dictionary)[1]
    
    for key,value in parsed_data.items():
        
        if key=='Supplier':
            Supplier=value['text']
        if key=='Legal Entity':
            Legal_Entity=value['text']
        if key=='PO Number':
            po_number=value['text']
        if key=='Invoice Date':
            invoice_date=value['text']
        if key=='Invoice Num':
            invoice_number=value['text']
        if key=='Invoice Curr':
            invoice_Curr=value['text']
        if key=='Line Total/Subtotal':
            subtotal=value['text']
        if key=='Invoice Amount':
            invoice_amount=value['text']
        if key=='Tax Rate':
            tax_rate=value['text']
        if key=='Tax Amount':
            tax_amount=value['text']
        if key=='Terms':
            terms=value['text']
        if key=='Service Start Date':
            start_date=value['text']
        if key=='Service End Date':
            end_date=value['text']
    return Supplier,Legal_Entity,po_number,invoice_date,invoice_number,invoice_Curr,subtotal,invoice_amount,tax_rate,tax_amount,terms,start_date,end_date

def data_extraction(dictionary):
    return dictionary['raw_text'],dictionary['pages'][0]['forms_data'],dictionary['pages'][0]['tables_data']
df['raw_text']=df['raw_json'].map(lambda s:data_extraction(s)[0])
df['forms_data']=df['raw_json'].map(lambda s:data_extraction(s)[1])
df['tables_data']=df['raw_json'].map(lambda s:data_extraction(s)[2])
#df['parsed_data']=df['raw_json'].map(lambda s:entities(s)[1])
df['Supplier']=df['raw_json'].map(lambda s:extract_data(s)[0])
df['Legal Entity']=df['raw_json'].map(lambda s:extract_data(s)[1])
df['PO Number']=df['raw_json'].map(lambda s:extract_data(s)[2]) 
df['Invoice Date']=df['raw_json'].map(lambda s:extract_data(s)[3]) 
df['Invoice Num']=df['raw_json'].map(lambda s:extract_data(s)[4]) 
df['Invoice curr']=df['raw_json'].map(lambda s:extract_data(s)[5]) 
df['Subtotal']=df['raw_json'].map(lambda s:extract_data(s)[6]) 
df['Invoice Amount']=df['raw_json'].map(lambda s:extract_data(s)[7]) 
df['Tax Rate']=df['raw_json'].map(lambda s:extract_data(s)[8]) 
df['Tax Amount']=df['raw_json'].map(lambda s:extract_data(s)[9])
df['Terms']=df['raw_json'].map(lambda s:extract_data(s)[10])
df['Service Start Date']=df['raw_json'].map(lambda s:extract_data(s)[11])
df['Service End Date']=df['raw_json'].map(lambda s:extract_data(s)[12])

*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926
*R
 8,926


In [149]:
df

,class_folder_path,raw_json,class_pred,raw_text,forms_data,tables_data,Supplier,Legal Entity,PO Number,Invoice Date,Invoice Num,Invoice curr,Subtotal,Invoice Amount,Tax Rate,Tax Amount,Terms,Service Start Date,Service End Date
0,027_TC26524856.json,{'raw_text': 'KRKM-§* EM@MX}EM) PORR11011–12 T...,1045939 - GOLDENCODE CO LTD,KRKM-§* EM@MX}EM)\nPORR11011–12 TC 26524856\nX...,"[{'fieldName': 'A: ', 'nameConfidence': 0.9273...","[{'idx': 1, 'confidence': 0.9847, 'coordinates...",89612904,91111080,453369,2021/12/24,26524856,NA,NA,NA,NA,"2,720",NA,NA,NA
1,YU31115624.json,{'raw_text': 'Uber Eats-UÊ htSith+U&#@]K=120%B...,1056012-HUIZHEN ENTERPRISE CORP,Uber Eats-UÊ\nhtSith+U&#@]K=120%B1\n0978388039...,"[{'fieldName': '*HẸVÃO ', 'nameConfidence': 0....","[{'idx': 1, 'confidence': 0.9189, 'coordinates...",70503981,Uber Eats,NA,2022-03-31 121104,31115624,NA,"8,926",NA,NA,4266,NA,20220303,20220330


In [150]:
df['raw_text'][1].split('\n')

['Uber Eats-UÊ',
 'htSith+U&#@]K=120%B1',
 '0978388039',
 'Z',
 '*R',
 '1-)',
 '8,926',
 '446',
 '9,372',
 '294071',
 '*HẸVÃO 20220303~20220330',
 'AHMW:F-@[318]##',
 '0506765',
 '***',
 '17',
 '11103-04',
 'YU-31115624',
 '2022-03-31 12:11:04 25',
 '6:4266',
 '9,372',
 '70503981',
 '91111080',
 '0506765',
 '0506765',
 'Page 1',
 'From:',
 'Date:',
 'To:',
 'Subject:',
 'Attachments:',
 '"Uber AP Taiwan" <ap-tw@uber.com>',
 'Fri, 1 Apr 2022 01:58:53 -0700',
 '"Kofax TW" <kofax-ap-tw@uber.com>',
 'PO #452514 † [R5X6D5-OPK5]',
 'Uber.pdf',
 '##- Please type your reply above this line -##',
 'A',
 'Apr 1, 2022, 10:58 AM GMT+2',
 '**',
 '***KOTRAJ',
 '2790-2146',
 '2790-2106',
 'pp9614@yahoo.com.tw',
 '***',
 'uberaf.zendesk.com/agent/tickets/2476870',
 '[R5X6D5-0PK5]',
 'You are an agent. Add a comment by replying to this email or view ticket in Zendesk Support',
 'Ticket # 2476870',
 'Status Pending',
 'Requester',
 'CCS Bwang20, Mike Her, Yyu20, Nyu4',
 'Group Uber AP Taiwan',
 'Assigne

In [107]:
#v=['Invoice Subtotal:']
for i in df['forms_data'][1]:
    #if i['fieldName']=='A:\n':
    print(i['fieldName']+"------------------"+i['fieldValue'])

*HẸVÃO ------------------20220303~20220330

ht------------------Sith+U&#@]K=120%B1

*R
------------------8,926

6:------------------4266

25
------------------2022-03-31 12:11:04 
YU-------------------31115624

Z
------------------1-)



In [79]:
for i in range(1,2):
    raw_text=df['raw_text'][i]
    p=raw_text.find('Monthly Recurring Fee')
    x = raw_text[p:p+70]
    desc_value=x.split('\n')[0].replace('-',' ')
    print(x,desc_value)
    pattern = re.findall('\w+\s\d+', desc_value)
    print(pattern)
    if pattern!=None:
        a,b=service_period_start_end_date(pattern[0])
    #line_value=x.split('\n')[4:7]
        print(a,b)
    #table_json={}
    #table_data=[]
    #print(desc_value,line_value)
    #for i,(des,line) in enumerate(zip(desc_value,line_value)):
        #print(i,des,line)
      #  table_json['Description'] = des
      #  table_json['Line_Total/Amount'] = line
      #  table_data.append(table_json)
       # print(table_data)
    #print(table_data)
    #print('@@@@@')



 
[]


NameError: name 'service_period_start_end_date' is not defined

In [143]:
for i in range(1,2):
    raw_text=df['raw_text'][i]
    p=raw_text.find('*HẸVÃO')
    x=raw_text[p:p+100]
    y=x.split('\n')[0].split(' ')[1]
    print(y.split('~'))

['20220303', '20220330']


In [224]:
def start_end_date_pattern_one(date_format):
    '''pattern = 01 Oct 2021 - 31 Oct 2021'''
    data = date_format.split('-')
    start_date = data[0].strip()
    end_date = data[1].strip()
    return (start_date, end_date)

def start_end_date_pattern_two(date_format):
    '''pattern = October 2021'''
    month_with_30_days_list = ["April", "June", "September", "November", "APRIL", "JUNE", "SEPTEMBER", "NOVEMBER","Apr", "Jun", "Sep", "Nov"]
    data = date_format.split(' ')
    month = data[0].strip()
    year = data[1].strip()
    start_date = '1'+'-'+month+'-'+year
    if month in month_with_30_days_list:
        end_date = "30" + "-" + month + "-" + year
    elif month == "February" or month == "FEBRUARY" or month=='Feb':
        if int(year) % 4 == 0:
            end_date = "29" + "-" + month + "-" + year
        else:
            end_date = "28" + "-" + month + "-" + year
    else:
        end_date = "31" + "-" + month + "-" + year
    
    return (start_date, end_date)

def start_end_date_pattern_three(date_format):
    ''' pattern = '5/31/2021'''
    month_with_30_days_list = ["4", "6", "9", "11"]
    data = date_format.split('/')
    month = data[0].strip()
    year = data[2].strip()
    start_date = '1'+'-'+month+'-'+year
    if month in month_with_30_days_list:
        end_date = "30" + "-" + month + "-" + year
    elif month == "February":
        if int(year) % 4 == 0:
            end_date = "29" + "-" + month + "-" + year
        else:
            end_date = "28" + "-" + month + "-" + year
    else:
        end_date = "31" + "-" + month + "-" + year
    
    return (start_date, end_date)

def start_end_date_pattern_four(date_format):
    '''pattern = July-Aug 2021'''
    month_with_30_days_list = ["April", "June", "September", "November", "APRIL", "JUNE", "SEPTEMBER", "NOVEMBER","Apr", "Jun", "Sep", "Nov"]
    data = date_format.split(' ')
    month = data[0].strip()
    split_month=month.split('-')
    year = data[1].strip()
    start_date = '1'+'-'+split_month[0]+'-'+year
    if split_month[1] in month_with_30_days_list:
        end_date = "30" + "-" + split_month[1] + "-" + year
    elif split_month[1] == "Feb" or split_month[1] == "FEB" or split_month[1] == "FEBRUARY":
        if int(year) % 4 == 0:
            end_date = "29" + "-" + split_month[1] + "-" + year
        else:
            end_date = "28" + "-" + split_month[1] + "-" + year
    else:
        end_date = "31" + "-" + split_month[1] + "-" + year
    
    return (start_date, end_date)
def service_period_start_end_date(data):
    '''Function to extract service start and end date'''
    try:
        if len(data)>20:
            start, end = start_end_date_pattern_one(data)
            print('1')
            return (start, end)
        elif re.fullmatch('\w+\s\d+',data):
            start, end = start_end_date_pattern_two(data)
            print('2')
            return (start, end)
        elif re.fullmatch('\d[/]\d+[/]\d{4}',data):
            start, end = start_end_date_pattern_three(data)
            print('3')
            return (start, end)
        elif re.fullmatch('\w+-\w+\s\d{4}',data):
            start, end = start_end_date_pattern_four(data)
            print('4')
            return (start, end)
        else:
            return('NA', 'NA')
    except:
        return('NA', 'NA')

    


In [38]:
raw_text=df['raw_text'][0]
p=raw_text.find('TC')
x=raw_text[p:p+100]
print(x.split('\n')[0])


TC 26524856


In [229]:
#df['class_folder_path'][83].split('/')[-1]